In [42]:
import os
import pandas as pd
import time
from collections import Counter
import bz2
import json
import re

In [43]:
def process_files(data_dir, output_file_path, can_reuse_output = True):
    if os.path.isfile(output_file_path) and can_reuse_output:
        return
    
    filenames = [filename for filename in os.listdir(data_dir) if filename.endswith('.json.bz2')]
    input_files_paths = [os.path.join(data_dir, filename) for filename in filenames]

    domain_matcher = re.compile(r"^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?(?P<domain>[^:\/?\n]+)")
    get_domain_from_url = lambda string: domain_matcher.match(string).group('domain')
        
    with bz2.open(output_file_path, 'wb') as output_file:
        for input_file_path in input_files_paths:
            start = time.time()
            
            with bz2.open(input_file_path, 'rb') as input_file:
                for i, line in enumerate(input_file):
                    line = json.loads(line)
                    
                    data_line = {'quote_word_count': Counter(line['quotation']),
                                 'speaker'         : line['speaker'],
                                 'qids'            : line['qids'],
                                 'first_date'      : line['date'],
                                 'domains'         : [get_domain_from_url(url) for url in line['urls']]}

                    output_file.write((json.dumps(data_line) + '\n').encode('utf-8'))
                    
                    if not i % 1000000:
                        print("Read", i, "lines from", input_file_path, 'in', (time.time() - start) / 60, "minutes")
                        
            print("Finished reading", input_file_path, 'in', (time.time() - start) / 60, "minutes")

In [ ]:
DATA_DIR = 'Data'
CACHE_DIR = 'Cache'

process_files(DATA_DIR, os.path.join(CACHE_DIR, 'processed_data.json.bz2'))

# POUR KAOU

Bonjour Kaou, tu as choisi word counting. Malheureusement je me suis rendu trop tard dans l'execution de la cellule précédente que j'avais oublié d'enlever les punctuation. Oups. Tu vas devoir itérer sur tous les clés du Compteur et enlever manuellement, et merge aussi les mêmes mots. Ou juste relancer la cellule du haut que j'ai déjà corrigé, mais que je n'ai pas eu le courage de relancer parce que ça prend beaucoup trop de temps (mais pas de resources, donc ne te fais pas de soucis tu peux le run sur n'importe quoi).

Lis cette page pour des idées sur comment faire évoluer ton travail au délà du word counting: https://scikit-learn.org/stable/modules/feature_extraction.html

Je te mets aussi à disposition une simple fonction qui efface les "English stop words" d'une liste de mots. Par contre, fais gaffe à comment tu l'utilises. Les stop words ne sont pas nécéssairement toutes toujours inutiles. Lis sklearn pour les détails et affiche la liste avant d'utiliser la fonction pour voir si elle t'arrange. Tu peux covertir la fonction pour qu'elle droppe le compte des stop words dans un compteur en faisant del counter_object[word].

In [6]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def remove_punctuation(string):
    return re.sub(r'[^a-z0-9 ]', '', string)

def remove_stopwords(word_list):
    return [word for word in word_list if word not in ENGLISH_STOP_WORDS]

# POUR CÉLINA

Coucou CC, il nous ont donné un file .parquet (juste un autre format de stockage binaire, un peux comme pickle) que normalement j'ai mis dans le dossier Data. Il devrait contenir plusieures informations sur chaque speaker. Tu peux le load comme ça, mais il me semble qu'il faut installer pyarrow (  conda install -c conda-forge pyarrow  ).

Malheureusement il parait qu'ils aient utilisé le QID de Wikidata pour encoder la profession, religion et tout le reste. Probablement une des première choses à faire est de trouver un moyen de mapper les QIDS à des string, que je sais faire en faisant des queries à wikidata mais je me demande s'il n'y a pas un moyen plus simple. Au pire on le fait (que une fois de toute façon).

Il faudrait aussi s'assurer que dans le parquet qu'ils nous donnent il n'y ait pas que peu d'occupations au bol, mais un grand nombre comme celui qu'on voit sur wikidata, car autrement on a le même problème que quand je faisais les queries (que j'ai découvert comment resudre) qui est que pour des gens avec beaucoup de professions on en avait seulement 3 au bol.

Oubliepas que je t'ai dit qu'il y a un problème avec l'antivirus et la lecture de .json. Je ne sais pas pourquoi, mais à chaque fois que tu veux commencer à lire un json tu dois mettre en standby l'anti-virus pendant une minute. C'est assez chiant, mais je n'ai pas trouvé d'autre solution (sinon, le programme ne plante pas, il s'arrete juste et ne dit rien et attend jusqu'à que tu desactives).

Pour le moment c'est tout je crois.

In [ ]:
pd.read_parquet('Data/speaker_attributes.parquet')

# POUR ANDREA
Bonjour Andrea, ça va? Oui très bien, merci. J'essaye de poser une base pour commencer le Milestone 2 du projet. Et toi? Moi aussi, drôle ça. Bon, à toute. Bon travail, à toute.

Il faut rerun la lecture du dataset avec la remotion de ponctuation.

Je suppose que tu vas faire la partie de correler les dates à des événements, et si t'as envie d'essayer d'extraire la variance. Par contre pour le moment je ne sais pas trop comment ça colle avec le reste de l'analyse. Dans le sens que la data story va parler de quoi concernant les dates?

# POUR MATTIA
Buongiorno Mattia, je suppose que tu vas faire la partie de regarder les newspapers et correler avec les speakers et, si on arrive, l'argument de la quote. Pour toi je crois que juste load le processed_data.json.bz2 et garder que les clés 'domains' et 'speaker' devrait le faire. Il faudra sûrement se coordonner avec Célina et Kaou pour voir justement comment correler les trucs. Pour le moment, je t'avoue que comme pour ma partie, je n'ai pas une idéé précide de comment ceci va coller dans une data story coherente. Faudra que Kaou et Célina avancent rapidement pour que tu puisse commencer à repliquer leur travail mais pour différents newspapers. Ou je ne sais pas. Faudra juste pas qu'on reste bloqués si on attend quelqu'un d'autre.

In [57]:
#Create a counter for the different domains of the urls
import bz2
import json
import dateutil.parser as dparser

input_file_path = 'quotes-2019-nytimes.json.bz2' 


domain=Counter()

start = time.time()

with bz2.open(input_file_path, 'rb') as input_file:
    for i, line in enumerate(input_file):
        line = json.loads(line)
        domain_matcher = re.compile(r"^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?(?P<domain>[^:\/?\n]+)")
        get_domain_from_url = lambda url: domain_matcher.match(url).group('domain')

        domain.update([get_domain_from_url(url) for url in line['urls']])


        if not i % 1000000:
            print("Read", i, "lines from", input_file_path, 'in', (time.time() - start) / 60, "minutes")

print("Finished reading", input_file_path, 'in', (time.time() - start) / 60, "minutes")

Read 0 lines from quotes-2019-nytimes.json.bz2 in 0.00034999847412109375 minutes
Finished reading quotes-2019-nytimes.json.bz2 in 0.28105957905451456 minutes


In [144]:
for k, v in domain.most_common(10):
    print ('%s: %i' % (k, v))

nytimes.com: 213819
msn.com: 45173
uspolitics.einnews.com: 40377
mobile.nytimes.com: 33893
pulse.com.gh: 20466
pulse.ng: 20404
krmg.com: 12298
wokv.com: 12260
seattletimes.com: 11959
breitbart.com: 11551


In [151]:
#Extract dates and look at the percentrage of extraction for total urls
input_file_path = 'quotes-2019-nytimes.json.bz2' 
output_file_path='test_dates.json.bz2'


matches=0
nomatch=0
total_urls=0
date=[]
start = time.time()
with bz2.open(output_file_path, 'wb') as output_file:
    with bz2.open(input_file_path, 'rb') as input_file:
        for i, line in enumerate(input_file):
            line = json.loads(line)
            total_urls+=len(line['urls'])
            dates=[]
            for url in line['urls']:
                match=re.search('\d{4}/\d{2}/\d{2}',url)
                if match is not None :
                    matches+=1
                    dates.append(match.group())
                    
            data_line = {'Dates': dates}

            output_file.write((json.dumps(data_line) + '\n').encode('utf-8'))                    
            if not i % 1000000:
                print("Read", i, "lines from", input_file_path, 'in', (time.time() - start) / 60, "minutes")

print("Finished reading", input_file_path, 'in', (time.time() - start) / 60, "minutes")

print("Percentage of date extraction: ",matches/total_urls*100)

Read 0 lines from quotes-2019-nytimes.json.bz2 in 0.00040004650751749676 minutes
Finished reading quotes-2019-nytimes.json.bz2 in 0.34906006256739297 minutes
Percentage of date extraction:  27.257588417657765
